<a href="https://colab.research.google.com/github/SeibelT/colab/blob/master/frommatlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Training a network with Keras**

 **Import libraries**

In [3]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D,Dropout

from keras.applications.resnet50 import ResNet50


from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras import Model,layers
from keras.optimizers import Adam,SGD,Adadelta
from sklearn.metrics import classification_report, confusion_matrix


Using TensorFlow backend.


# Import Data

To be able to import Data in google Colaboratory, we have to connect to My_Drive first

In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount("My_Drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at My_Drive


If not working try "Reset all Runtime" in Runtime-menu or try following code(remove hashtags first):

# Datagen for training, validation and testing

target_size equal to Model input_size


In [3]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
     'My_Drive/My Drive/TransferedMatlabcode/data_rearranged/training',
    target_size=(224,224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

validation_generator = validation_datagen.flow_from_directory(
     'My_Drive/My Drive/TransferedMatlabcode/data_rearranged/validation',
    target_size=(224,224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=True)






Found 3496 images belonging to 8 classes.
Found 760 images belonging to 8 classes.


# Prepare Model

**Load Resnet without top layers and create own model**

In [0]:
base_model=ResNet50(weights='imagenet',include_top=False) #imports the Restnet50 model without last layers.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(128,activation='relu')(x) #add some dense layers, Dropoutlayers(less overfitting)
x=Dropout(0.3)(x)
x=Dense(128,activation='relu')(x) #dense layer 2
x=Dropout(0.3)(x)
x=Dense(64,activation='sigmoid')(x) #dense layer 3
preds=Dense(8,activation='softmax')(x) #final layer with softmax 
model=Model(inputs=base_model.input,outputs=preds)


#show Layers from new model
#for i,layer in enumerate(model.layers):
#  print(i,layer.name)

# Train Model

with different Optimizer
select optimizer, make if-Function True, others False

In [0]:
if True:
  """Adam Optimizer"""
  ADAM=keras.optimizers.Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, amsgrad=False)
  model.compile(loss='categorical_crossentropy',optimizer=ADAM,metrics=['accuracy'])
  # Adam optimizer
  # loss function will be categorical cross entropy, since dataset has more categories
  # evaluation metric will be accuracy

if False:
  """SGD Optimizer"""
  sgd=keras.optimizers.SGD(lr=0.001)
  model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
if False:
  """Adadelta optimizer"""
  adadelt= keras.optimizers.Adadelta(lr=0.1)
  model.compile(loss='categorical_crossentropy',optimizer=adadelt,metrics=['accuracy'])

  


"""train model,(later use history for confusion matrix)"""

step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=validation_generator.n//train_generator.batch_size

history=model.fit_generator(generator=train_generator,
                    validation_data=validation_generator, validation_steps=step_size_valid, validation_freq=1,
                   steps_per_epoch=step_size_train,
                   epochs=15
                   )




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/15
 31/109 [=======>......................] - ETA: 15:53 - loss: 1.6015 - acc: 0.4929

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 37 could not be retrieved. It could be because a worker has died.
  UserWarning)


 91/109 [========================>.....] - ETA: 3:38 - loss: 1.1334 - acc: 0.7170

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 32 could not be retrieved. It could be because a worker has died.
  UserWarning)


 95/109 [=========================>....] - ETA: 2:54 - loss: 1.1173 - acc: 0.7240

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 8 could not be retrieved. It could be because a worker has died.
  UserWarning)


102/109 [===========================>..] - ETA: 1:30 - loss: 1.0956 - acc: 0.7316

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 20 could not be retrieved. It could be because a worker has died.
  UserWarning)


109/109 [==============================] - 1587s 15s/step - loss: 1.0738 - acc: 0.7382 - val_loss: 0.9931 - val_acc: 0.7269
Epoch 2/15
  1/109 [..............................] - ETA: 2:34 - loss: 1.1982 - acc: 0.5000

# Save Model
if necessary 

In [0]:
#architecture and weights to HDF5
#model.save('My_Drive/My Drive/TransferedMatlabcode/model_sgd_resnet50_50epochs.h5')

#print(model.summary())

# Create Datagen for testing model

In [4]:

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)



test_generator= test_datagen.flow_from_directory(
     'My_Drive/My Drive/TransferedMatlabcode/data_rearranged/test',
    target_size=(224,224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=False)



FileNotFoundError: ignored

# Confusion Matrix 


In [0]:
#print(history.history.keys())

num_of_test_samples=760
batch_size=32

pred = model.predict_generator(test_generator)
Pred = np.argmax(pred, axis=1) #scale 0-1
print('simple Confusion Matrix:')
print(confusion_matrix(test_generator.classes, Pred))
print('Classification Report:')
target_names = ["Tumor", "Stroma", "Complex", "Lympho", "Debris" ,"Mucosa", "Adipose", "Empty"]

#classification report (?)
print(classification_report(test_generator.classes, Pred, target_names=target_names))


# Plot Accuracy & Loss

In [0]:
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, constrained_layout=True)

ax1.plot(history.history['acc'],label="training")
ax1.plot(history.history['val_acc'],label="validation")
ax1.axhline(max(history.history['val_acc']),color="grey",label="max val accuracy")
ax1.set_title("Accuracy ")
ax1.set_ylabel("accuracy")

ax1.legend()
ax2.plot(history.history['loss'],label="training")
ax2.plot(history.history['val_loss'],label="validation")
ax2.set_xlabel("epochs")
ax2.set_ylabel("loss")
ax2.legend()


"""to save fig as pdf:"""
fname="My_Drive/My Drive/TransferedMatlabcode/plot_training_resnet_Adadelta"
#plt.savefig(fname, format="pdf")
